In [1]:
suppressPackageStartupMessages({
    library(tidymodels)
    library(treesnip)
    library(tidyverse)
    library(keras)
    library(tensorflow)
    #library(lime)
})

In [2]:
seed <- 1923

In [62]:
set.seed(1)

label <- 'default'

iris1.5 <- ISLR::Default %>% mutate(default = case_when(default == 'No' ~ 0,
                                             default == 'Yes' ~ 1))

iris1.5 <- iris1.5 %>% mutate(default = factor(default,levels = 0:1))

willsee <- caret::createDataPartition(iris1.5[[label]],p = .7,list = TRUE) %>% unlist

iris2 <- iris1.5[willsee,]

iris3 <- iris1.5[-willsee,]

folds_ <- caret::createFolds(iris2[[label]],k = 5)

folds <- 1:length(folds_)

errors <- c()

predictions <- list()

iris2 <- iris2 %>% mutate(default = as.integer(as.character(default)))
iris3 <- iris3 %>% mutate(default = as.integer(as.character(default)))

i <- 1

train_indices_dum <- setdiff(folds,i)
test_indices_dum <- i

train_set_dum <- iris2[unlist(folds_[train_indices_dum]),]
test_set_dum <- iris2[unlist(folds_[test_indices_dum]),]


In [63]:
input_neurons <- ncol(train_set_dum) - 1
output_neurons <- 2

nofsample <- nrow(train_set_dum)

alpha <- 4

In [64]:
numofneuron <- nofsample / (alpha * (input_neurons + output_neurons))

numofneuron <- numofneuron - output_neurons

In [65]:
layer_dist1 <- base::seq(from = numofneuron,to = output_neurons,length.out = 10)
layer_dist <- layer_dist1[-1]/base::sum(layer_dist1[-1])*numofneuron
layer_disto <- layer_dist %>% ceiling %>% .[1:8]

#randy <- sample(1:1000,size = 8)

#(numofneuron*(randy/sum(randy)) )%>% ceiling

In [66]:
layer_disto

[1] 62 54 47 39 31 24 16  9

In [67]:
f1 <- function(y_true, y_pred){
    y_pred = k_round(y_pred)
    
    tp = k_sum(k_cast(y_true*y_pred, 'float'), axis=1)
    tn = k_sum(k_cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = k_sum(k_cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = k_sum(k_cast(y_true*(1-y_pred), 'float'), axis=1)

    p = tp / (tp + fp + k_epsilon())
    r = tp / (tp + fn + k_epsilon())

    f1 = 2*p*r / (p+r+k_epsilon())
    f1 = tf$where(tf$math$is_nan(f1), tf$zeros_like(f1), f1)
    return(k_mean(f1,axis = 1))
}

f1_loss <- function(y_true, y_pred){
    #y_pred = k_round(y_pred)
    tp = k_sum(k_cast(y_true*y_pred, 'float'), axis=1)
    tn = k_sum(k_cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = k_sum(k_cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = k_sum(k_cast(y_true*(1-y_pred), 'float'), axis=1)

    p = tp / (tp + fp + k_epsilon())
    r = tp / (tp + fn + k_epsilon())

    f1 = 2*p*r / (p+r+k_epsilon())
    f1 = tf$where(tf$math$is_nan(f1), tf$zeros_like(f1), f1)
    return(1 - k_mean(f1,axis = 1))
}

In [68]:
#model %>% compile(
#  loss = f1_loss,
#  optimizer = optimizer_adam(learning_rate = 0.01),
#  metrics = c(f1)
#)

In [78]:
pls <- list()

for(i in folds){
    train_indices <- setdiff(folds,i)
    test_indices <- i
    
    train_set <- iris2[unlist(folds_[train_indices]),]
    test_set <- iris2[unlist(folds_[test_indices]),]
    
    tarif <- recipe(default ~ .,data = train_set) %>% 
    step_normalize(all_numeric_predictors()) %>%
    #step_encode_mean(all_nominal_predictors(),outcome = 'default') %>%
    step_dummy(all_nominal_predictors()) %>%
    prep
    
    target_data <- bake(tarif,iris3)
    
    train_set <- bake(tarif,train_set)
    test_set <- bake(tarif,test_set)
    
    train_y <- train_set[,label] %>% as.matrix %>% to_categorical
    test_y <- test_set[,label]%>% as.matrix %>% to_categorical
    test_y2 <- test_set[,label]%>% as.matrix
    
    train_x <- train_set[, -which(names(train_set) == label)]%>% as.matrix
    test_x <- test_set[, -which(names(test_set) == label)]%>% as.matrix
    
    target_data <- target_data[, -which(names(train_set) == label)]%>% as.matrix
    
    tf$random$set_seed(seed = seed)

    reticulate::py_set_seed(seed = seed)

    model <- keras_model_sequential()
    
    randy <- sample(1:1000,size = 8)

    unites <- (numofneuron*(randy/sum(randy)) )%>% ceiling

    model %>%
    layer_dense(units = layer_disto[1], activation = 'relu', input_shape = c(3)) %>%
    layer_dense(units = layer_disto[2], activation = 'relu') %>%
    layer_dense(units = layer_disto[3], activation = 'relu') %>%
    layer_dense(units = layer_disto[4], activation = 'relu') %>%
    layer_dense(units = layer_disto[5], activation = 'relu') %>%
    layer_dense(units = layer_disto[6], activation = 'relu') %>%
    layer_dense(units = layer_disto[7], activation = 'relu') %>%
    layer_dense(units = layer_disto[8], activation = 'relu') %>%
    layer_dense(units = 2,activation = 'softmax')
    
    model %>% compile(
      loss = f1_loss,
      optimizer =optimizer_rmsprop(learning_rate = 0.0001),
      metrics = c(f1)
    )
    
    wgh <- sqrt(sum(test_y)/sum(test_y == 0))
    
    history <- model %>% fit(
      train_x, train_y, 
      epochs = 1500, batch_size = 128, 
      validation_data = list(test_x,test_y),
      callbacks = list(callback_early_stopping(
                        monitor = "val_loss",
                        patience = 10,
                        verbose = 1,
                        mode = "auto",
                        restore_best_weights = TRUE
                        )
    ),
    class_weight = list('0' = wgh,'1' = 1)
    )

    preds <- model %>% predict(test_x)  %>% 
                        as.data.frame %>%
                        mutate('class'=names(.)[apply(., 1, which.max)]) %>%
                        mutate(class = case_when(class == 'V1' ~ 0,
                                                 class == 'V2' ~ 1)) %>%
                        select(class) %>%
                        pull
    
    error <- yardstick::f_meas_vec(truth = as.factor(test_y2),estimate = factor(preds,levels = 0:1),event_level = "second")
    
    preds_target <- model %>% predict(target_data)  %>% 
                        as.data.frame %>%
                        mutate('class'=names(.)[apply(., 1, which.max)]) %>%
                        mutate(class = case_when(class == 'V1' ~ 0,
                                                 class == 'V2' ~ 1)) %>%
                        select(class) %>%
                        pull
    
    IRdisplay::display('**************----------**************')
    IRdisplay::display(sprintf('Fold %s F1 : %s',i,round(error,2)))
    errors[i] <- error
    
    pls[[i]] <- history %>% plot
    
    predictions[[i]] <- preds_target
}
IRdisplay::display('**************----------**************')
IRdisplay::display(sprintf('CV Mean F1 : %s',mean(errors,na.rm = T)))

dson <- bake(tarif,iris2)

strain_y <- dson[,label] %>% as.matrix %>% to_categorical

strain_x <- dson[, -which(names(dson) == label)]%>% as.matrix
    

historyson <-  model %>% fit(
      strain_x, strain_y, 
      epochs = 100, batch_size = 128, 
      validation_data = list(test_x,test_y),
      callbacks = list(callback_early_stopping(
                        monitor = "val_loss",
                        patience = 100,
                        verbose = 1,
                        mode = "auto",
                        restore_best_weights = TRUE
                        )
    ),
    class_weight = list('0' = wgh,'1' = 1)
    )

[1] "**************----------**************"

[1] "Fold 1 F1 : 0.54"

[1] "**************----------**************"

[1] "Fold 2 F1 : 0.54"

[1] "**************----------**************"

...

[1] "Fold 4 F1 : 0.58"

[1] "**************----------**************"

[1] "Fold 5 F1 : 0.64"

[1] "**************----------**************"

[1] "CV Mean F1 : 0.540525240663052"

In [79]:
#historyson %>% plot

In [80]:
#pls %>% lapply(function(x) x + xlim(0,150))

In [81]:
#dsson <- bake(tarif,iris3)

#sstrain_y <- dsson[,label] %>% as.matrix %>% to_categorical

#sstrain_x <- dsson[, -which(names(dsson) == label)]%>% as.matrix

#sstrain_y2 <- dsson[,label] %>% as.matrix 

In [82]:
#prds <- predict(model,sstrain_x) %>% apply(1,which.max) - 1

In [83]:
#sstrain_y2 %>%
#as.data.frame %>% 
#bind_cols(a = prds) %>%
#mutate_all(as.factor) %>%
#f_meas(truth = default,estimate = a,event_level = "second")

In [84]:
all <- as.data.frame(predictions) %>% mutate_if(is.factor,function(x) as.numeric(as.character(x)))

In [85]:
kk <- rowMeans(all) 
kk <- ifelse(kk >= 0.5,1,0)

In [86]:
kl <- iris3[,label]

In [87]:
keras_preds <- kk

In [88]:
yardstick::f_meas_vec(truth = as.factor(kl),estimate = factor(kk,levels = 0:1),event_level = "second")

[1] 0.4820513

In [89]:
set.seed(1)

label <- 'default'

iris1.5 <- ISLR::Default %>% mutate(default = case_when(default == 'No' ~ 0,
                                             default == 'Yes' ~ 1))

iris1.5 <- iris1.5 %>% mutate(default = factor(default,levels = 0:1))

willsee <- caret::createDataPartition(iris1.5[[label]],p = .7,list = TRUE) %>% unlist

iris2 <- iris1.5[willsee,]

iris3 <- iris1.5[-willsee,]

folds_ <- caret::createFolds(iris2[[label]],k = 5)

folds <- 1:length(folds_)

errors <- c()

predictions <- list()

#iris2 <- iris2 %>% mutate(default = as.integer(as.character(default)))
#iris3 <- iris3 %>% mutate(default = as.integer(as.character(default)))

for(i in folds){
    train_indices <- setdiff(folds,i)
    test_indices <- i
    
    train_set <- iris2[unlist(folds_[train_indices]),]
    test_set <- iris2[unlist(folds_[test_indices]),]
    
    tarif2 <- recipe(default ~ .,data = train_set)
    
    target_data <- bake(tarif,iris3)
    
    train_set <- bake(tarif %>% prep,train_set)
    test_set <- bake(tarif%>% prep,test_set)
    
    train_y <- train_set[,label]
    test_y <- test_set[,label]
    test_y2 <- test_set[,label]%>% as.matrix
    
    train_x <- train_set[, -which(names(train_set) == label)]
    test_x <- test_set[, -which(names(test_set) == label)]
    
    target_data <- target_data[, -which(names(train_set) == label)]
    
    modspec <- boost_tree(learn_rate = 0.01) %>%
    set_mode('classification')%>%
    set_engine('catboost',auto_class_weights = 'SqrtBalanced',
               eval_metric = 'F1:use_weights=False',
               custom_loss = 'F1:use_weights=False')
    
    
    model <- modspec %>% fit_xy(x = train_x,y = train_y)
    
    preds <- model %>% predict(test_x) %>% pull
    
    error <- yardstick::f_meas_vec(truth = as.factor(test_y2),estimate = factor(preds,levels = 0:1),event_level = "second")
    
    preds_target <- model %>% predict(target_data) %>%pull
    
    IRdisplay::display('**************----------**************')
    IRdisplay::display(sprintf('Fold %s F1 : %s',i,round(error,2)))
    errors[i] <- error
    
    predictions[[i]] <- preds_target
}
IRdisplay::display('**************----------**************')
IRdisplay::display(sprintf('CV Mean F1 : %s',mean(errors)))

[1] "**************----------**************"

[1] "Fold 1 F1 : 0.51"

[1] "**************----------**************"

[1] "Fold 2 F1 : 0.52"

[1] "**************----------**************"

...

[1] "Fold 4 F1 : 0.58"

[1] "**************----------**************"

[1] "Fold 5 F1 : 0.62"

[1] "**************----------**************"

[1] "CV Mean F1 : 0.529256025691767"

In [90]:
all <- as.data.frame(predictions) %>% mutate_if(is.factor,function(x) as.numeric(as.character(x)))

In [91]:
kk <- rowMeans(all) 
kk <- ifelse(kk >= 0.5,1,0)

In [92]:
kl <- iris3[,label]

In [93]:
catboost_preds <- kk

In [94]:
yardstick::f_meas_vec(truth = as.factor(kl),estimate = factor(kk,levels = 0:1),event_level = "second")

[1] 0.4880383

In [36]:
library(targencoding)

In [55]:
tarif <- recipe(default ~ . ,ISLR::Default %>% mutate(default = case_when(default == 'Yes' ~ 1, default == 'No' ~ 0))) %>%
step_encode_mean(all_nominal_predictors(),outcome = 'default') %>%prep

In [56]:
juice(tarif)

student,balance,income,default
<dbl>,<dbl>,<dbl>,<dbl>
0.02919501,729.5265,44361.625,0
0.04313859,817.1804,12106.135,0
0.02919501,1073.5492,31767.139,0
0.02919501,529.2506,35704.494,0
0.02919501,785.6559,38463.496,0
0.04313859,919.5885,7491.559,0
0.02919501,825.5133,24905.227,0
0.04313859,808.6675,17600.451,0
0.02919501,1161.0579,37468.529,0


In [33]:
comp <- data.frame(keras = keras_preds,catboost = catboost_preds,orgi = kl)

In [7]:
library(dbplyr)
library(dplyr)

In [2]:
tb <- data.frame(year = 1,mo=1,da=1)

In [15]:
tb_l <- tbl_lazy(tb,con = simulate_mssql())

In [16]:
tb_l %>%
mutate(Date = as.Date(paste0(year,mo,day)))

<SQL>
SELECT *, TRY_CAST(`year` + `mo` + `day` AS DATE) AS `Date`
FROM `df`

In [1]:
library(xgboost)

In [23]:
iris2 <- iris[,-5]

In [25]:
ind <- as.numeric(caret::createDataPartition(iris2[,4],times = 1,p = .7,list = F))#%>% as.numeric

In [111]:
make_class <- function(x) ifelse(x >= 2.2,1,0)

In [112]:
x <- iris2[,-4]
y <- make_class(iris2[,4])

In [113]:
x_train <- x[ind,]
y_train <- y[ind]

x_test <- x[-ind,]
y_test <- y[-ind]

In [114]:
dt <- xgb.DMatrix(as.matrix(x_train),label = y_train)

In [115]:
dt_test <- xgb.DMatrix(as.matrix(x_test))

In [181]:
pr <- list(objective = 'binary:logistic',eta = .01)

In [182]:
psk <- sum(y_train == 0)/sum(y_train == 1)

In [183]:
modk <- xgb.train(data = dt,nrounds = 200,verbose = 1,params = pr,scale_pos_weight = sqrt(psk))

In [184]:
prop.table(table(y_train))

y_train
        0         1 
0.8962264 0.1037736 

In [185]:
prop.table(table(ifelse(predict(modk,dt_test)>=.5,1,0)))


        0         1 
0.8409091 0.1590909 

In [188]:
prop.table(table(y_test))

y_test
        0         1 
0.8636364 0.1363636 